In [ ]:
#hide
from fileresponse.core import *

# Welcome to django_fileresponse

> Serve files directly from Django.

`django_fileresponse` is a library that allows you to serve files directly from Django.

## Features of django_fileresponse

`django_fileresponse` provides the following features for developers:

- **Use asyncio to serve files with high concurrency** directly from Django.
- Uses [aiofiles](https://github.com/Tinche/aiofiles) to **asynchronously read from filesystem** and [aiobotocore](https://github.com/aio-libs/aiobotocore) to **asynchronously read from s3 compatible object stores**

## Installing

`django_fileresponse` is on PyPI so you can just run `pip install django_fileresponse`.

## How to use

Fill me in please! Don't forget code examples:

In [ ]:
1+1

2